In [1]:
'''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [3]:
NUM_VERTIHUBS = 2000 #dependent on trip_data
FLIGHT_SPEED = 60 #m/s

In [4]:
'''
Trip data formatting
'''
trip_data = pd.read_csv('data/Realtime/OpsLimits/trips_16.csv')
trip_data.drop(trip_data[trip_data.OriginVertihub == trip_data.DestinationVertihub].index, inplace=True) #drop trips where the origin and destination are the same vertiport, reduced from 4801 trips to 1692 with 10 vertiports
trip_data

,Time,OriginVertihub,OriginVertiport,DestinationVertihub,DestinationVertiport
1,0,9,18,7,99
3,60,4,137,14,83
4,60,9,119,3,38
5,60,6,124,1,49
7,0,9,67,1,88
...,...,...,...,...,...
9488,0,14,145,0,73
9491,60,3,61,9,32
9495,0,9,115,11,118
9496,0,3,114,14,108


In [5]:
vertiports_data = pd.read_csv('data/Realtime/OpsLimits/vertiports_2000.csv')
vertiports_data

,vertiport,long,lat,vertihub
0,0,-88.077616,41.691089,1750
1,1,-88.335512,42.284315,136
2,2,-87.590294,41.736803,1962
3,3,-87.867464,42.013719,1767
4,4,-89.143889,42.128561,372
...,...,...,...,...
1995,1995,-87.906844,41.775018,1513
1996,1996,-87.897864,42.095067,1900
1997,1997,-87.568446,41.593811,248
1998,1998,-87.801232,41.536060,360


In [6]:
'''
vertiport statistics
'''
vertiports_per_vertihub = [0 for i in range(NUM_VERTIHUBS)]
for index, vertiport in vertiports_data.iterrows():
    vertiports_per_vertihub[int(vertiport['vertihub'])] += 1
assert(sum(vertiports_per_vertihub) == vertiports_data.shape[0])
vertiports_per_vertihub



[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [7]:
from math import cos, asin, sqrt, pi

# def dist(x1, y1, x2, y2):
#     return math.sqrt((y2-y1)**(y2-y1) + (x2-x1)**(x2-x1))
def ll_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...   
    

In [13]:
'''
trip statistics
'''

arrivals_per_vertihub = [[] for i in range(NUM_VERTIHUBS)]
flight_popularity = dict()
# get the trip arrival time per vertihub
for index, trip in trip_data.iterrows():
    #calculate time of arrival (begin at trip.time), calculated between vertiports
    origin_port = vertiports_data.iloc[trip['OriginVertiport']]
    destination_port = vertiports_data.iloc[trip['DestinationVertiport']]
    if((origin_port['vertiport'],destination_port['vertiport']) not in flight_popularity.keys()):
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] = 1
    else:
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] += 1
    #trip distance in kilometers
    dist = ll_distance(origin_port['lat'], origin_port['long'], destination_port['lat'], destination_port['long'])
    travel_time = (dist*1000)/FLIGHT_SPEED # (km * 1000)/(m/s) -> seconds
    arrival_time = trip['Time'] + travel_time #takeoff time + travel_time = arrival time
    #add arrival time to vertihub
    arrivals_per_vertihub[trip['DestinationVertihub']].append(arrival_time)

# for each list of vertihub, calculate the minimum, average, and maximum distance between arrivals
differences_per_vertihub = []
# simulated_arrivals
for arrivals_list in arrivals_per_vertihub:
    sorted_arrivals = sorted(arrivals_list)
    differences = []
    if(len(sorted_arrivals) <= 1):
        continue
    for index in range(len(sorted_arrivals)-1):
        differences.append(sorted_arrivals[index+1] - sorted_arrivals[index])
    # print(sorted_arrivals)
    differences_per_vertihub.append(sorted(differences))

for index, differences in enumerate(differences_per_vertihub):
    print('vertihub ' + str(index) + ' has minimum arrival difference of ' + str(differences[0]))
    print('vertihub ' + str(index) + ' has average arrival difference of ' + str(sum(differences)/len(differences)))
    print('vertihub ' + str(index) + ' has maximum arrival difference of ' + str(differences[len(differences)-1]))
    print('vertihub ' + str(index) + ' has ' + str(len(differences)) + ' trips')
    print('')
   

formatted_flight_popularity = [(flight_popularity[key], key) for key in flight_popularity.keys()]
print('Most popular flight path ' + str((sorted(formatted_flight_popularity))[len(formatted_flight_popularity)-1]))
print('Total flight paths ' + str(len(formatted_flight_popularity)))
avg = sum([num for num, thing in formatted_flight_popularity])/len(formatted_flight_popularity)
print('Mean requests per flight path ' + str(avg))
formatted_vertihub_popularity = [(len(differences_per_vertihub[i]), i) for i in range(len(differences_per_vertihub))]

print(sorted(formatted_vertihub_popularity))
    
    

vertihub 0 has minimum arrival difference of 0.0
vertihub 0 has average arrival difference of 19.156600464229435
vertihub 0 has maximum arrival difference of 224.30482344640677
vertihub 0 has 164 trips

vertihub 1 has minimum arrival difference of 0.0
vertihub 1 has average arrival difference of 15.586409891706701
vertihub 1 has maximum arrival difference of 413.22754551896014
vertihub 1 has 217 trips

vertihub 2 has minimum arrival difference of 0.0
vertihub 2 has average arrival difference of 71.8362952082885
vertihub 2 has maximum arrival difference of 558.4582633963864
vertihub 2 has 40 trips

vertihub 3 has minimum arrival difference of 0.0
vertihub 3 has average arrival difference of 3.636253203726348
vertihub 3 has maximum arrival difference of 98.87788473973615
vertihub 3 has 795 trips

vertihub 4 has minimum arrival difference of 0.0
vertihub 4 has average arrival difference of 14.680673720993376
vertihub 4 has maximum arrival difference of 583.2152142875912
vertihub 4 has 265